In [2]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json
img_width, img_height = 224, 224

Using TensorFlow backend.


In [14]:
def mag_conn(mag):
    train_dir = 'breast_' + mag + '_split/train'
    val_dir = 'breast_' + mag + '_split/val'
    test_dir = 'breast_' + mag + '_split/test'

    if mag == '40':
        b_train, m_train, b_test, m_test, b_val, m_val = 500, 1170, 100, 137, 137, 174
    if mag == '100':
        b_train, m_train, b_test, m_test, b_val, m_val = 515, 1237, 100, 145, 135, 188
    if mag == '200':
        b_train, m_train, b_test, m_test, b_val, m_val = 498, 1190, 100, 139, 137, 178
    if mag == '400':
        b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147        
    
    nb_train =  b_train + m_train
    nb_val =  b_val + m_val
    nb_test =  b_test + m_test

    train_labels = np.array([0] * b_train + [1] * m_train)
    val_labels = np.array([0] * b_val + [1] * m_val)
    test_labels = np.array([0] * b_test + [1] * m_test)
    return train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels

In [18]:
def fine_extract():
    train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
    validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')
        
    tuned_model_new_name = 'tuned_TL_' + mag_fact

    # load json and create model
    json_file = open(tuned_model_new_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    tuned_model = model_from_json(model_json)
    # load weights into new model
    tuned_model.load_weights(tuned_model_new_name + '.h5')
    print("Loaded model from disk")

    # evaluate loaded model on test data
    tuned_model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])

    intermediate_from_model = tuned_model.get_layer("block5_pool").output
    bottleneck_model = Model(tuned_model.input, intermediate_from_model)
    bottleneck_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


    fine_bottleneck_feat_train = bottleneck_model.predict_generator(train_generator, nb_train//batch_size)
    np.save(open('fine_bottleneck_feat_train_' + mag_fact +'.npy', 'wb'), fine_bottleneck_feat_train)
    print("train extracted")

    fine_bottleneck_feat_val = bottleneck_model.predict_generator(validation_generator, nb_val//batch_size)
    np.save(open('fine_bottleneck_feat_val_' + mag_fact +'.npy', 'wb'), fine_bottleneck_feat_val)
    print("validation extracted")

    fine_bottleneck_feat_test = bottleneck_model.predict_generator(test_generator, nb_test//batch_size)
    np.save(open('fine_bottleneck_feat_test_' + mag_fact +'.npy', 'wb'), fine_bottleneck_feat_test)
    print("test extracted")

In [17]:
def BACH_fine_extract(mag):
    bach_b_test = 200
    bach_m_test = 200
    nb_bach_test =  bach_b_test + bach_m_test

    model_name = 'tuned_TL_' + mag

    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")

    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])

    intermediate_from_model = model.get_layer("block5_pool").output
    bottleneck_model = Model(model.input, intermediate_from_model)
    bottleneck_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    
    bach_test_dir = 'BACH/'
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        bach_test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    fine_bottleneck_feat_bach = bottleneck_model.predict_generator(
        test_generator, nb_bach_test//batch_size)
    np.save(open('fine_bottleneck_feat_bach_' + mag +'.npy', 'wb'), fine_bottleneck_feat_bach)
    print("BACH extracted")

    score = model.evaluate_generator(test_generator,test_generator.samples//batch_size)
    print('\n', 'BACH Test accuracy:', score[1])


In [16]:
mag_fact = '40'
batch_size = 1
train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag_fact)
fine_extract()

Found 1670 images belonging to 2 classes.
Found 311 images belonging to 2 classes.
Found 237 images belonging to 2 classes.
Loaded model from disk


In [19]:
mag_fact = '100'
batch_size = 1
train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag_fact)
fine_extract()

Found 1752 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 245 images belonging to 2 classes.
Loaded model from disk
train extracted
validation extracted
test extracted


In [20]:
mag_fact = '200'
batch_size = 1
train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag_fact)
fine_extract()

Found 1688 images belonging to 2 classes.
Found 315 images belonging to 2 classes.
Found 239 images belonging to 2 classes.
Loaded model from disk
train extracted
validation extracted
test extracted


In [26]:
mag_fact = '400'
batch_size = 1
train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag_fact)
fine_extract()

Found 1502 images belonging to 2 classes.
Found 287 images belonging to 2 classes.
Found 224 images belonging to 2 classes.
Loaded model from disk
train extracted
validation extracted
test extracted


In [22]:
mag_fact = '40'
BACH_fine_extract(mag_fact)

Loaded model from disk
Found 400 images belonging to 2 classes.
BACH extracted

 BACH Test accuracy: 0.4925000071525574


In [23]:
mag_fact = '100'
BACH_fine_extract(mag_fact)

Loaded model from disk
Found 400 images belonging to 2 classes.
BACH extracted

 BACH Test accuracy: 0.5024999976158142


In [24]:
mag_fact = '200'
BACH_fine_extract(mag_fact)

Loaded model from disk
Found 400 images belonging to 2 classes.
BACH extracted

 BACH Test accuracy: 0.5099999904632568


In [27]:
mag_fact = '400'
BACH_fine_extract(mag_fact)

Loaded model from disk
Found 400 images belonging to 2 classes.
BACH extracted

 BACH Test accuracy: 0.5
